#Descrizione per generare date

In [0]:
"""
Il tipo date di databricks da documentazione supporta il seguente range:
The range of dates supported is June 23 -5877641 CE to July 11 +5881580 CE.
Source: https://docs.databricks.com/aws/en/sql/language-manual/data-types/date-type

Per la generazione delle date utilizza quindi come base di partenza il datatype DATE di databricks, ecco qualche esempio:
-------
> SELECT DATE'0000';
  0000-01-01

> SELECT DATE'2020-12-31';
  2020-12-31

> SELECT DATE'2021-7-1T';
  2021-07-01

> SELECT cast('1908-03-15' AS DATE)
  1908-03-15

> SELECT DATE'-10000-01-01'
  -10000-01-01
--------

Considera ora questo range piu' ridotto: dal 3999 a.C. al 2100 d.C...

considera le seguenti date:
	• la minima data supportata dal range ridotto
	• l'ultima data appena precedente alla nascita di cristo
	• la data della nascita di cristo
	• la prima data appena successiva alla nascita di cristo
	• la data di oggi
	• la massima data supportata dal range ridotto

voglio che crei una tabella chiamata special_dates e la popoli, per ogni data indicata sopra, con i seguenti campi:
	• era (campo utilizzato per identificare se si riferisce all’era pre-cristiana (BCE) o all’era attuale (CE)),
	• data in formato date,
	• Iso_date (data in formato iso)
	• millennio,
	• secolo,
	• secolo_name,
	• decennio,
	• anno,
	• quarter (i quarter durano 3 mesi, sono 4 in un anno e il primo quarter inizia a gennaio)
	• quarter_name,
	• Stagione_num (considera le effettive date di cambio stagione),
	• Stagione_nome (considera le effettive date di cambio stagione),
	• mese,
	• mese_nome,
	• settimana
	• settimana_nome,
	• giorno,
	• giorno_nome,
	• anno mese (in formato int),
	• anno mese giorno (in formato int),
	• flg_leap_year (identifica se e' un anno bisestile),
	• flg_weekend (identifica se e' un fine settimana),
	• Descrizione

i nomi dei campi devono essere in inglese.
crea due statement distinti:
	• Lo statement di creazione tabella,
	• Lo statement di inserimento 
"""

# Date Limite

In [0]:
%sql
USE CATALOG dad_open_data;
USE SCHEMA time;

In [0]:
%sql
-- Create the special_dates table
CREATE OR REPLACE TABLE dad_open_data.time.special_dates (
  era               STRING,
  dt                DATE,
  iso_date          STRING,
  millennium        INT,
  century           INT,
  century_name      STRING,
  decade            INT,
  year_num          INT,
  quarter           INT,
  quarter_name      STRING,
  season_num        INT,
  season_name       STRING,
  month_num         INT,
  month_name        STRING,
  week_num          INT,
  week_name         STRING,
  day_num           INT,
  day_name          STRING,
  year_month        INT,
  year_month_day    INT,
  flg_leap_year     BOOLEAN,
  flg_weekend       BOOLEAN,
  description       STRING
);

-- Insert the six special dates
INSERT INTO dad_open_data.time.special_dates
SELECT
  CASE WHEN year(dt) <= 0 THEN 'BCE' ELSE 'CE' END                                            AS era,
  dt                                                                                         AS dt,
  CAST(dt AS STRING)                                                                         AS iso_date,
  CASE
    WHEN year(dt) >= 1 THEN FLOOR((year(dt) - 1) / 1000) + 1
    ELSE -FLOOR(ABS(year(dt)) / 1000)
  END                                                                                         AS millennium,
  CASE
    WHEN year(dt) >= 1 THEN FLOOR((year(dt) - 1) / 100) + 1
    ELSE -FLOOR(ABS(year(dt)) / 100)
  END                                                                                         AS century,
  CONCAT(CAST(
    CASE
      WHEN year(dt) >= 1 THEN FLOOR((year(dt) - 1) / 100) + 1
      ELSE -FLOOR(ABS(year(dt)) / 100)
    END AS STRING), ' Century')                                                               AS century_name,
  FLOOR(year(dt) / 10) * 10                                                                  AS decade,
  year(dt)                                                                                   AS year_num,
  quarter(dt)                                                                                AS quarter,
  CONCAT('Q', CAST(quarter(dt) AS STRING))                                                   AS quarter_name,
  CASE
    WHEN (month(dt) = 12 AND day(dt) >= 21) OR (month(dt) IN (1,2)) OR (month(dt) = 3 AND day(dt) < 20) THEN 1
    WHEN (month(dt) = 3 AND day(dt) >= 20) OR (month(dt) IN (4,5)) OR (month(dt) = 6 AND day(dt) < 21) THEN 2
    WHEN (month(dt) = 6 AND day(dt) >= 21) OR (month(dt) IN (7,8)) OR (month(dt) = 9 AND day(dt) < 22) THEN 3
    ELSE 4
  END                                                                                         AS season_num,
  CASE
    WHEN (month(dt) = 12 AND day(dt) >= 21) OR (month(dt) IN (1,2)) OR (month(dt) = 3 AND day(dt) < 20) THEN 'Winter'
    WHEN (month(dt) = 3 AND day(dt) >= 20) OR (month(dt) IN (4,5)) OR (month(dt) = 6 AND day(dt) < 21) THEN 'Spring'
    WHEN (month(dt) = 6 AND day(dt) >= 21) OR (month(dt) IN (7,8)) OR (month(dt) = 9 AND day(dt) < 22) THEN 'Summer'
    ELSE 'Autumn'
  END                                                                                         AS season_name,
  month(dt)                                                                                  AS month_num,
  date_format(dt, 'MMMM')                                                                    AS month_name,
  weekofyear(dt)                                                                             AS week_num,
  CONCAT('Week ', CAST(weekofyear(dt) AS STRING))                                            AS week_name,
  day(dt)                                                                                    AS day_num,
  date_format(dt, 'EEEE')                                                                    AS day_name,
  year(dt) * 100 + month(dt)                                                                 AS year_month,
  year(dt) * 10000 + month(dt) * 100 + day(dt)                                               AS year_month_day,
  CASE
    WHEN (year(dt) % 4 = 0 AND year(dt) % 100 != 0) OR (year(dt) % 400 = 0) THEN TRUE
    ELSE FALSE
  END                                                                                         AS flg_leap_year,
  CASE WHEN dayofweek(dt) IN (1,7) THEN TRUE ELSE FALSE END                                 AS flg_weekend,
  description                                                                                AS description
FROM (
  SELECT DATE'-3999-01-01'   AS dt, 'Minimum supported date'                     AS description UNION ALL
  SELECT DATE'0000-12-31'    AS dt, 'Last day before the birth of Christ'      AS description UNION ALL
  SELECT DATE'0001-01-01'    AS dt, 'Birth of Christ (day 1 CE)'               AS description UNION ALL
  SELECT DATE'0001-01-02'    AS dt, 'First day after the birth of Christ'      AS description UNION ALL
  SELECT CURRENT_DATE()      AS dt, 'Today'                                    AS description UNION ALL
  SELECT DATE'2100-12-31'    AS dt, 'Maximum supported date'                   AS description
) AS src;

In [0]:
%sql
select description, * 
from special_dates
order by dt asc

#Tutte le date

In [0]:
%sql
-- Create the special_dates table
CREATE OR REPLACE TABLE dad_open_data.time.dates (
  era               STRING,
  dt                DATE,
  iso_date          STRING,
  millennium        INT,
  century           INT,
  century_name      STRING,
  decade            INT,
  year_num          INT,
  quarter           INT,
  quarter_name      STRING,
  season_num        INT,
  season_name       STRING,
  month_num         INT,
  month_name        STRING,
  week_num          INT,
  week_name         STRING,
  day_num           INT,
  day_name          STRING,
  year_month        INT,
  year_month_day    INT,
  flg_leap_year     BOOLEAN,
  flg_weekend       BOOLEAN,
  description       STRING
);

-- Insert all dates from 3999 BCE to 2100 CE, preserving the six original descriptions
INSERT INTO dad_open_data.time.dates
WITH
  -- all dates in the required range
  all_dates AS (
    SELECT explode(
             sequence(
               to_date('-3999-01-01'),      -- 3999 BCE
               to_date('2100-12-31'),       -- 2100 CE
               interval 1 day
             )
           ) AS dt
  ),
  -- the six special dates with their descriptions
  special_desc AS (
    SELECT DATE'-3999-01-01' AS dt, 'Minimum supported date'                     AS description UNION ALL
    SELECT DATE'0000-12-31'  AS dt, 'Last day before the birth of Christ'      AS description UNION ALL
    SELECT DATE'0001-01-01'  AS dt, 'Birth of Christ (day 1 CE)'               AS description UNION ALL
    SELECT DATE'0001-01-02'  AS dt, 'First day after the birth of Christ'      AS description UNION ALL
    SELECT CURRENT_DATE()   AS dt, 'Today'                                    AS description UNION ALL
    SELECT DATE'2100-12-31' AS dt, 'Maximum supported date'                   AS description
  )
SELECT
  CASE WHEN year(a.dt) <= 0 THEN 'BCE' ELSE 'CE' END                                            AS era,
  a.dt                                                                                         AS dt,
  CAST(a.dt AS STRING)                                                                         AS iso_date,
  CASE
    WHEN year(a.dt) >= 1 THEN FLOOR((year(a.dt) - 1) / 1000) + 1
    ELSE -FLOOR(ABS(year(a.dt)) / 1000)
  END                                                                                         AS millennium,
  CASE
    WHEN year(a.dt) >= 1 THEN FLOOR((year(a.dt) - 1) / 100) + 1
    ELSE -FLOOR(ABS(year(a.dt)) / 100)
  END                                                                                         AS century,
  CONCAT(CAST(
    CASE
      WHEN year(a.dt) >= 1 THEN FLOOR((year(a.dt) - 1) / 100) + 1
      ELSE -FLOOR(ABS(year(a.dt)) / 100)
    END AS STRING), ' Century')                                                               AS century_name,
  FLOOR(year(a.dt) / 10) * 10                                                                  AS decade,
  year(a.dt)                                                                                   AS year_num,
  quarter(a.dt)                                                                                AS quarter,
  CONCAT('Q', CAST(quarter(a.dt) AS STRING))                                                   AS quarter_name,
  CASE
    WHEN (month(a.dt) = 12 AND day(a.dt) >= 21) OR (month(a.dt) IN (1,2)) OR (month(a.dt) = 3 AND day(a.dt) < 20) THEN 1
    WHEN (month(a.dt) = 3 AND day(a.dt) >= 20) OR (month(a.dt) IN (4,5)) OR (month(a.dt) = 6 AND day(a.dt) < 21) THEN 2
    WHEN (month(a.dt) = 6 AND day(a.dt) >= 21) OR (month(a.dt) IN (7,8)) OR (month(a.dt) = 9 AND day(a.dt) < 22) THEN 3
    ELSE 4
  END                                                                                         AS season_num,
  CASE
    WHEN (month(a.dt) = 12 AND day(a.dt) >= 21) OR (month(a.dt) IN (1,2)) OR (month(a.dt) = 3 AND day(a.dt) < 20) THEN 'Winter'
    WHEN (month(a.dt) = 3 AND day(a.dt) >= 20) OR (month(a.dt) IN (4,5)) OR (month(a.dt) = 6 AND day(a.dt) < 21) THEN 'Spring'
    WHEN (month(a.dt) = 6 AND day(a.dt) >= 21) OR (month(a.dt) IN (7,8)) OR (month(a.dt) = 9 AND day(a.dt) < 22) THEN 'Summer'
    ELSE 'Autumn'
  END                                                                                         AS season_name,
  month(a.dt)                                                                                  AS month_num,
  date_format(a.dt, 'MMMM')                                                                    AS month_name,
  weekofyear(a.dt)                                                                             AS week_num,
  CONCAT('Week ', CAST(weekofyear(a.dt) AS STRING))                                            AS week_name,
  day(a.dt)                                                                                    AS day_num,
  date_format(a.dt, 'EEEE')                                                                    AS day_name,
  year(a.dt) * 100 + month(a.dt)                                                               AS year_month,
  year(a.dt) * 10000 + month(a.dt) * 100 + day(a.dt)                                           AS year_month_day,
  CASE
    WHEN (year(a.dt) % 4 = 0 AND year(a.dt) % 100 != 0) OR (year(a.dt) % 400 = 0) THEN TRUE
    ELSE FALSE
  END                                                                                         AS flg_leap_year,
  CASE WHEN dayofweek(a.dt) IN (1,7) THEN TRUE ELSE FALSE END                                 AS flg_weekend,
  d.description                                                                                AS description
FROM all_dates a
LEFT JOIN special_desc d
  ON a.dt = d.dt;

In [0]:
%sql
SELECT description, *
FROM dates
WHERE year_num = YEAR(CURRENT_DATE())
ORDER BY dt DESC

# KPI by date

In [0]:
"""
Per ogni data presente nella tabella dates, calcola I seguenti kpis e salvali in una tabella dates_kpi:

• Numero del quarter
• Somma cumulata del numero dei quarter nell'anno

• Numero mese nell'anno (es. Da 1 a 12)
• Somma cumulata del numero di mesi nell'anno
• Numero mese nel quarter (es. Da 1 a 3)
• Somma cumulata del numero di mesi del quarter
• 
• Numero settimana nell'anno (es. Da 1 a 52)
• Somma cumulata del numero di settimane nell'anno
• Numero settimana nel quarter
• Somma cumulata del numero di settimane nel quarter
• Numero settimana nel mese
• Somma cumulata del numero di settimane nel mese
• 
• Numero del giorno nell'anno (es. Da 1 a 365) 
• Somma cumulata dei numeri di giorno nell'anno
• Numero del giorno nel quarter
• Somma cumulata dei numeri di giorni nel quarter
• Numero del giorno nel mese
• Somma cumulata dei numeri di giorni nel mese
• Numero del giorno nella settimana
Somma cumulata dei numeri di giorni nella settimana
"""

In [0]:
%sql
-- Create the KPI table
CREATE OR REPLACE TABLE dad_open_data.time.dates_kpi (
  dt                     DATE,
  quarter_num            INT,
  cum_quarter_in_year    INT,
  month_num              INT,
  cum_month_in_year      INT,
  month_in_quarter       INT,
  cum_month_in_quarter   INT,
  week_num               INT,
  cum_week_in_year       INT,
  week_in_quarter        INT,
  cum_week_in_quarter    INT,
  week_in_month          INT,
  cum_week_in_month      INT,
  day_of_year            INT,
  cum_day_of_year        INT,
  day_in_quarter         INT,
  cum_day_in_quarter     INT,
  day_in_month           INT,
  cum_day_in_month       INT,
  day_in_week            INT,
  cum_day_in_week        INT
);

-- Populate the KPI table
INSERT INTO dad_open_data.time.dates_kpi
SELECT
  dt,
  quarter(dt)                                            AS quarter_num,
  SUM(quarter(dt)) OVER (PARTITION BY year(dt) ORDER BY dt
                         ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_quarter_in_year,
  month(dt)                                              AS month_num,
  SUM(month(dt)) OVER (PARTITION BY year(dt) ORDER BY dt
                       ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_month_in_year,
  ((month(dt) - 1) % 3) + 1                               AS month_in_quarter,
  SUM(((month(dt) - 1) % 3) + 1) OVER (PARTITION BY year(dt), quarter(dt) ORDER BY dt
                                      ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_month_in_quarter,
  weekofyear(dt)                                         AS week_num,
  SUM(weekofyear(dt)) OVER (PARTITION BY year(dt) ORDER BY dt
                            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_week_in_year,
  FLOOR(DATEDIFF(dt, DATE(date_trunc('quarter', dt))) / 7) + 1 AS week_in_quarter,
  SUM(FLOOR(DATEDIFF(dt, DATE(date_trunc('quarter', dt))) / 7) + 1)
    OVER (PARTITION BY year(dt), quarter(dt) ORDER BY dt
          ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_week_in_quarter,
  FLOOR(DATEDIFF(dt, DATE(date_trunc('month', dt))) / 7) + 1   AS week_in_month,
  SUM(FLOOR(DATEDIFF(dt, DATE(date_trunc('month', dt))) / 7) + 1)
    OVER (PARTITION BY year(dt), month(dt) ORDER BY dt
          ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_week_in_month,
  dayofyear(dt)                                            AS day_of_year,
  SUM(dayofyear(dt)) OVER (PARTITION BY year(dt) ORDER BY dt
                           ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_day_of_year,
  (dayofyear(dt) - dayofyear(DATE(date_trunc('quarter', dt))) + 1) AS day_in_quarter,
  SUM(dayofyear(dt) - dayofyear(DATE(date_trunc('quarter', dt))) + 1)
    OVER (PARTITION BY year(dt), quarter(dt) ORDER BY dt
          ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_day_in_quarter,
  day(dt)                                                 AS day_in_month,
  SUM(day(dt)) OVER (PARTITION BY year(dt), month(dt) ORDER BY dt
                     ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_day_in_month,
  dayofweek(dt)                                           AS day_in_week,
  SUM(dayofweek(dt)) OVER (PARTITION BY year(dt), weekofyear(dt) ORDER BY dt
                           ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_day_in_week
FROM dad_open_data.time.dates
WHERE day_num = 1;

In [0]:
%sql
SELECT    description, 
          d.*,
          k.*
FROM      dates AS d
left join dates_kpi AS k
    on    d.dt = k.dt
WHERE     year_num = YEAR(CURRENT_DATE())
ORDER BY  d.dt DESC